In [1]:
with open('./data/input_10.txt') as fh:
    file_input = fh.read().strip()

In [2]:
test1= """.#..#
.....
#####
....#
...##"""

In [3]:
test2 = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##"""

In [4]:
import numpy as np
from itertools import combinations
from collections import defaultdict, Counter

In [5]:
def parse(txt):
    lines = txt.splitlines()
    coords = []
    for i, line in enumerate(lines):
#         print (i, line)
        coords += [(j, i) for j, c in enumerate(line) if c=='#']
    return coords

def path(c1, c2):
    dx = c2[0] - c1[0]
    xdiff = abs(dx)
    dy = c2[1] - c1[1]
    ydiff = abs(dy)
    steps = max(xdiff, ydiff)
    dx = 1. * dx / steps
    dy = 1. * dy / steps
#     print(steps, dx, dy)
    for i in range(1, steps):
        x = (c1[0]+dx*i)
        y = (c1[1]+dy*i)
        if x == int(x) and y == int(y):
            yield int(x), int(y)

def visible(coords):
    blocked = set(coords)
    visible = defaultdict(list)
    for pair in combinations(coords, 2):
        if not set(path(*pair)) & blocked:
            visible[pair[0]] += [pair[1]]
            visible[pair[1]] += [pair[0]]
    return visible

def find_max(vis):
    _m = 0
    _c = (0, 0)
    for c in vis:
        _l = len(vis[c])
        if _l > _m:
            _m = _l
            _c = c
    return _c, _m

In [6]:
# test
coords = parse(test1)
vis = visible(coords)
find_max(vis)

((3, 4), 8)

In [7]:
# test 2
coords = parse(test2)
vis = visible(coords)
find_max(vis)

((11, 13), 210)

In [8]:
# part 1
coords = parse(file_input)
vis = visible(coords)
find_max(vis)

((37, 25), 309)

In [57]:
# part 2

def diff(a, b):
    dx = b[0] - a[0]
    dy = b[1] - a[1]
    alpha = (np.degrees(np.arctan2(dy, dx)) * -1) # 90)
    
        
    return alpha

def candidates(coords):
    vis = visible(coords)
#     print(len(vis))
    c, n = find_max(vis)
    cand = np.array(vis[c]) - np.array(c)
    ang = np.degrees(np.arctan2(cand[:, 0], cand[:, 1]))
    dist = np.abs(cand).sum(axis=1)
    order = np.array(list(zip(ang, dist)), dtype=[('degrees', '<f4'), ('dist', '<f4')])
    while True:
        idx = order.argsort(axis=0, order=('degrees', 'dist'))
        idx = idx[::-1]
        blocked = np.diff(order['degrees'][idx], append=0) == 0
        if not blocked.any():
            break
        order['degrees'][idx] -= 360 * blocked
    return cand[idx] + c


In [61]:
candidates(parse(test2))[199]

array([8, 2])

In [64]:
x, y = candidates(parse(file_input))[199]
print(x*100 + y)

416
